## Homework 4

In this notebook I use principal component analysis in conjunction with K-nearest neighbors to classify the LSTM dataset.

In [1]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.neighbors import KNeighborsClassifier

test = pd.read_csv("~/Documents/Math352/hw4/test.csv")
train = pd.read_csv("~/Documents/Math352/hw4/train.csv")

x_train, y_train = train.iloc[:,1:].values, train.iloc[:,0].values
x_test = test.iloc[:,0:].values

In [2]:
train.head()

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Fit scaler to training data and transform it.

In [5]:
#scale data
sc = StandardScaler()
x_train_std = sc.fit_transform(x_train.astype('float64'))
x_test_std = sc.transform(x_test.astype('float64'))

Get the covariance matrix, obtain eigen values and calculate the cumulative variance explained ratio

In [6]:
cov_mat = np.cov(x_train_std.T)

#obtain eigenvalues, eigenvectors
eigen_vals, eigen_vecs = np.linalg.eig(cov_mat)


#compute variance explained ratio
total = sum(eigen_vals)
var_exp = eigen_vals/total  
eigen_vals.sort()
cum_var_exp = np.cumsum(var_exp) #calculate the cum sum of explained variance


To get the cumulative amount of variance explained, iterate over cum_var_exp while the value is less than the amount of variance we want.

In [29]:
def get_n_item_cum_var(var_exp, perc_explained):
    item = 0
    for i in var_exp:
        if i > perc_explained:
            return item
        item += 1
    

Define a function that incorporates the number of components determined by PCA and then calls KNN using regular euclidean distance. Beware, it's really slow.

In [28]:
def pca_knn(num_components, num_neighbors, x_train, x_test, y_train, per_explained):
    
    pca = PCA(n_components = num_components)
    knn = KNeighborsClassifier(n_neighbors = num_neighbors, p = 2, metric = 'minkowski')
    x_train_pca_sklearn = pca.fit_transform(x_train)
    x_test_pca_sklearn = pca.transform(x_test_std)
    knn.fit(x_train_pca_sklearn, y_train)
    

    y_pred_sklearn = knn.predict(x_test_pca_sklearn)
    
    # used to write to a file to submite to kaggle

In [ ]:
pca_knn(num_components = get_n_item_cum_var(cum_var_exp, .9), num_neighbors = 3, x_train = x_train_std, 
        x_test = x_test_std, y_train = y_train, per_explained = .90)
pca_knn(num_components = get_n_item_cum_var(cum_var_exp, .95), num_neighbors = 3, x_train = x_train_std, 
        x_test = x_test_std, y_train = y_train, per_explained = .95)
pca_knn(num_components = get_n_item_cum_var(cum_var_exp, .99), num_neighbors = 3, x_train = x_train_std, 
        x_test = x_test_std, y_train = y_train, per_explained = .99)

232
